## Primer 3. Maximum Coverage

Funkcije maksimalne pokrivenosti skupa (eng. *maximum coverage functions*) imaju cilj da maksimizuju broj atributa elemenata razlicitih od 0.
Podaci su predstavljeni u vidu binarne matrice gde $1$ znači da je stavka prisutna u elementu $x \in X$, a $0$ da nije.
Ove funkcije su korisne kada je prostor variabli valik, a svaki element skupa sadrži samo mali podskup njih - sto je čest slučaj kada se analiziraju tekstualni podaci, gde su variable reci.

Funckija maksimalne pokrivenosti je funkcija zasnovana na atributima (eng *feature based function*), gde je funkcija $\phi$ *minimum*.
Napomena: Sve vrednosti skupa podataka moraju biti binarne da bi apricot selektor radio.

Opšti oblik funkcije pokrivenosti je: <br />
$f(X)=\displaystyle \sum_{d=1}^{D} min(\displaystyle \sum_{i=1}^{N} X_{i,d},1)$ <br />

gde je $f$ funkcija nad podskupom $X$ koji ima $N$ elemenata i $D$ dimenzija.

Naredni primer je primer iz Apricot zvanične dokumentacije. <br />
Optimizujemo *max coverage* funkciju nad skupom od 11228 novinskih članaka *[reuters](https://keras.io/api/datasets/reuters/)*. Svaki novinski članak je sačuvan kao lista indeksa reči. Indeks je dodeljen reči u zavisnosti od njene učestalosti u kompletnom skupu reči. Na primer, indeksom 3 je određena treća najčešća reč iz skupa.  <br />

*Maximum coverage* funkcija računa broj jedinstvenih reči u tekstu. Rezultujući podskup $X$ sadrži $n$ izdvojenih novinskih članaka, tako da je broj jedinstvenih reči u njima najveći mogući.

In [1]:
from apricot import MaxCoverageSelection
from keras.datasets import reuters
import numpy as np

In [2]:
(X_, _), (_, _) = reuters.load_data(num_words=5000)

In [3]:
print('X_.shape: ', X_.shape)

X_.shape:  (8982,)


In [4]:
X_[:1]

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])],
      dtype=object)

In [5]:
# Učitavam mapiranje reči sa njihovim indeksima.
word_map = reuters.get_word_index(path='reuters_word_index.json')

# Modifikujem mapu tako da je mapiranje indeks -> reč.
word_map_inverse = {v: k for k, v in word_map.items()}

In [6]:
# Ispisujem reci koje pripadaju prvom članku učitanog skupa podataka.
for word_index in X_[:1][0]:
    #print(word_index)
    print(word_map_inverse[word_index], end=" ")

the of of mln loss for plc said at only ended said commonwealth could 1 traders now april 0 a after said from 1985 and from foreign 000 april 0 prices its account year a but in this mln home an states earlier and rise and revs vs 000 its 16 vs 000 a but 3 psbr oils several and shareholders and dividend vs 000 its all 4 vs 000 1 mln agreed largely april 0 are 2 states will billion total and against 000 pct dlrs 

Generišem binarnu matricu $X$ na osnovu učitanih podataka.  <br />
Vrste nove matrica $X$ označavaju novinske članke, indeks reda predstavlja reč, tj. mapirani indeks reči.
Vrednost $X[i][j]$ $0$ ili $1$ označava da li se data $j$ nalazi u članku $i$ ili ne.

In [7]:
X = np.zeros((X_.shape[0], max(map(max, X_))+1))
for i, x in enumerate(X_):
    X[i][x] = 1

In [8]:
max(max(X_))

4992

In [9]:
X[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [10]:
print('X.shape: ', X.shape)

X.shape:  (8982, 5000)


In [11]:
# Iskoristicu posebnu podrsku za retke matrice
from scipy.sparse import csr_matrix

X_sparse = csr_matrix(X)

In [12]:
n = 250
model = MaxCoverageSelection(n, optimizer='naive')
X_transformed = model.fit_transform(X_sparse)

In [13]:
print('Ukupan broj izdvojenih jedinstvenih reci: ', model.gains[:n].sum())

Ukupan broj izdvojenih jedinstvenih reci:  4949.0


Koriscenjem podrske za retke matrice optimizacija se ubrzava skoro 3x.

In [14]:
X_transformed.shape

(250, 5000)

In [15]:
# Kreiram standardnu matricu od CSR matrice.
X_transformed_dense = X_transformed.todense()
X_transformed_dense.shape

(250, 5000)

Ispisujem skup jedinstvenih reči iz izabranih n članaka.

In [16]:
selected_unique_word_indices = set()
for i in range(X_transformed_dense.shape[0]):
    for j in range(X_transformed_dense.shape[1]):
        if X_transformed_dense[i, j] == 1:
            selected_unique_word_indices.add(j)
print('Ukupan broj jedinstvenih reči u rezultujućem skupu: ', len(selected_unique_word_indices))

Ukupan broj jedinstvenih reči u rezultujućem skupu:  4949
